# Andmete puhastamine

Siin töötleme varasemas etapis kogutud toorandmeid (`course_details_merged.csv`).
Faili leiad projekti ÜHISEST repositooriumist. 

**Sisend:** `../data/toorandmed_aasta.csv`

**Väljund:** `../data/andmed_aasta.csv`

### Samm 0: Teekide laadimine ja failiteede seadistamine 
Laeme vajalikud Pythoni teegid ja fikseerime sisend- ja väljundfailide asukohad. Loeme sisse puhastamata andmed.

In [2]:
import pandas as pd
import json
import numpy as np
import os

# Konfigureerime Pandase sätteid loetavuse huvides
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Sisend- ja väljundfailid
INPUT_FILE = 'data/toorandmed_aasta.csv'
OUTPUT_FILE = '../data/andmed_aasta.csv'


if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE, low_memory=False)
    print(f"Toorandmed loetud. Andmestiku suurus: {df.shape[0]} rida, {df.shape[1]} veergu.")
else:
    # Kui faili pole, loome testimiseks tühja DataFrame'i või viskame vea
    raise FileNotFoundError(f"Viga: Sisendfaili '{INPUT_FILE}' ei leitud! Kontrolli failiteed.")

# Kiire pilk andmestruktuurile
df.head(2)

print("Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.")

Toorandmed loetud. Andmestiku suurus: 3416 rida, 229 veergu.
Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.


### Samm 1: eemaldame andmed, mida me ei taha

Siin võib filtreerida välja ridu mõne tunnuse alusel kui teame, et oma rakenduses vajame ainult teatud tüüpi andmeid, hetkel me seda ei teinud, aga kui tahaksime nt ainult kevade aineid, saaksime seda siin teha.

### Samm 2: Puhastatud veergude lisamine

Valime välja huvipakkuvad veerud ning puhastame need. Antud sammu juures seisneb puhastamine õige veeru valimises - juhul kui huvipakkuv sisu võib olla kahes erinevas veerus, eelistame aine konkreetse versiooni infot.


Täpsem kirjeldus aine üldine info vs versiooni info probleemist:
* **Probleem:** Meil on topelt veerge eeldatavasti sama infoga (üldine aine info vs aine konkreetse versiooni info). Nt eestikeelne pealkiri võiks peituda nii `title__et` kui ka `version__title__et` veerus.
* **Lahendus:** Eelistame alati `version__` prefiksiga veerge, sest need kajastavad tegelikku aineprogrammi. Kui see puudub, võtame üldise info.

In [5]:
def resolve_fields(df):
    """
    Loob uued puhtad veerud, eelistades versiooni-põhist infot.
    Tagastab täiendatud DataFrame'i.
    """

    # 1. Veerud, kus ühendame üldise info ja versiooni info (eelistades versiooni)
    # (uus nimi, üldise info veerg, versiooni info veerg)
    merge_mapping = [
        ('nimi_et', 'title__et', 'version__title__et'),
        ('nimi_en', 'title__en', 'version__title__en'),
        ('eap', 'credits', 'version__credits'),
        ('kestus_sem', 'additional_info__duration_in_semesters', 'version__additional_info__duration_in_semesters'),
        ('kirjeldus', 'overview__description__et', 'version__overview__description__et'),
        ('kirjeldus_en', 'overview__description__en', 'version__overview__description__en'),
        ('opivaljundid_et', 'overview__learning_outcomes_text_et', 'version__overview__learning_outcomes_text_et'),
        ('opivaljundid_en', 'overview__learning_outcomes_text_en', 'version__overview__learning_outcomes_text_en'),
        ('eeltingimused', 'additional_info__prerequisites', 'version__additional_info__prerequisites'),
        ('hindamisskaala_et', 'additional_info__assessment_scale__et', 'version__additional_info__assessment_scale__et'),
    ]

    # 2. Veerud, mis võetakse ühest konkreetsest kohast (lihtne ümbernimetamine)
    # (uus nimi, vana nimi)
    rename_mapping = [
        ('aine_kood', 'code'),
        ('semester', 'version__target__semester__et'),
        ('oppekeel_et', 'version__target__language__et'),
        ('oppekeel_en', 'version__target__language__en'),
        ('oppevorm_et', 'version__target__study_type__et'),
        ('oppeaste', 'version__additional_info__study_levels'),
        ('linn', 'version__target__course_main_structural_unit__city'),
        ('aine_tyyp_et', 'general__type__et'),
        ('oppejoud_json', 'version__participants__lecturers'),
        ('toimumisajad_json', 'version__schedule__entries'),
        ('hindamine_et', 'version__grading__grade_evaluation__et'),
        ('oppekeeled', 'version__overview__study_languages')
    ]

    print("Alustan veergude filtreerimist ja ühendamist...")
    
    for new_col, base, version in merge_mapping:
        base_exists = base in df.columns if base else False
        ver_exists = version in df.columns if version else False
        
        if base_exists and ver_exists:
            df[new_col] = df[version].fillna(df[base])
        elif ver_exists:
            df[new_col] = df[version]
        elif base_exists:
            df[new_col] = df[base]
        else:
            df[new_col] = np.nan
    for new_col, source in rename_mapping:
        if source in df.columns:
            df[new_col] = df[source]
        else:
            df[new_col] = np.nan
    return df


# Rakendame funktsiooni
df_resolved = resolve_fields(df.copy())

# Kontrollime tulemust
print(f"Samm 2: Puhastatud veergude lisamine tehtud.")
print(df_resolved[['aine_kood', 'semester']].isnull().sum())
print(df_resolved['semester'].unique())

Alustan veergude filtreerimist ja ühendamist...
Samm 2: Puhastatud veergude lisamine tehtud.
aine_kood     0
semester     17
dtype: int64
['kevad' 'sügis' nan]



### Samm 3: JSON väljade parsimine

See koodiplokk tegeleb tehniliste JSON-formaadis veergude teisendamisega inimloetavaks tekstiks. Eesmärk on eraldada keerulisest struktuurist vaid oluline sisu (nt nimed või nädalapäevad) ja vormistada see komadega eraldatud loeteluks.

Näide teisendamisest (õppejõud):
* Enne (Toores JSON): ``'[{"person_name": "Mari Mets", "id": 101}, {"person_name": "Jaan Kask", "id": 102}]'``
* Pärast (Puhastatud): ``"Jaan Kask, Mari Mets"``

In [6]:
# --- Abifunktsioonid JSON töötlemiseks ---

def parse_json_safe(json_str):
    """
    Teisendab JSON-stringi turvaliselt Pythoni objektiks (list või dict).
    Käsitleb tühje väärtusi (NaN, None) ja katkist JSON-it, tagastades vea korral None.
    """
    if pd.isna(json_str) or json_str == '': 
        return None
    try:
        return json.loads(json_str)
    except (json.JSONDecodeError, TypeError):
        return None


def extract_lecturers(json_str):
    """
    Eraldab JSON-struktuurist õppejõudude nimed ('person_name').
    Tagastab komadega eraldatud stringi unikaalsetest nimedest tähestikulises järjekorras.
    """
    data = parse_json_safe(json_str)
    if not data: return None
    names = [p.get('person_name') for p in data if isinstance(p, dict) and p.get('person_name')]
    return ", ".join(sorted(list(set(names)))) if names else None
 

def extract_schedule_days_et(json_str):
    """
    Analüüsib tunniplaani aegu ja leiab, mis nädalapäevadel aine toimub.
    1. Parsib kuupäeva/kellaaja.
    2. Teisendab päeva nime eesti keelde (nt Monday -> Esmaspäev).
    3. Sorteerib päevad loogilises järjekorras (Esmaspäevast Pühapäevani), mitte tähestiku järgi.
    """
    data = parse_json_safe(json_str)
    if not data: return None
    
    days = set()
    day_map = {
        'Monday': 'Esmaspäev', 'Tuesday': 'Teisipäev', 'Wednesday': 'Kolmapäev',
        'Thursday': 'Neljapäev', 'Friday': 'Reede', 'Saturday': 'Laupäev', 'Sunday': 'Pühapäev'
    }
    
    for entry in data:
        if not isinstance(entry, dict): continue
        time_str = entry.get('time') or entry.get('start_time')
        if time_str:
            dt = pd.to_datetime(time_str, errors='coerce')
            if not pd.isna(dt):
                en_day = dt.day_name()
                days.add(day_map.get(en_day, en_day))
    
    week_order = ['Esmaspäev', 'Teisipäev', 'Kolmapäev', 'Neljapäev', 'Reede', 'Laupäev', 'Pühapäev']
    sorted_days = sorted(list(days), key=lambda d: week_order.index(d) if d in week_order else 99)
    
    return ", ".join(sorted_days) if sorted_days else None


# --- Funktsioonide rakendamine ---
print("Ekstraheerime JSON väljadest infot...")

df_resolved['oppejoud'] = df_resolved['oppejoud_json'].apply(extract_lecturers)
df_resolved['toimumisajad'] = df_resolved['toimumisajad_json'].apply(extract_schedule_days_et)
  
print("Samm 3: JSON töötlemine valmis.")
print(df_resolved[['aine_kood', 'oppejoud', 'toimumisajad']].head(3))


Ekstraheerime JSON väljadest infot...
Samm 3: JSON töötlemine valmis.
     aine_kood                  oppejoud        toimumisajad
0  OIEO.06.046                 Gea Lepik                None
1  FLFI.00.016  Alexander Stewart Davies                None
2  KKSB.05.092          Kadri Medijainen  Neljapäev, Laupäev


### Samm 4 (Valikuline): Info koondamine mitmest allikast

Kui vajalik info on mitmes veerus, saame need ühendada. Hetkel teeme seda näiteks hindamisinfoga.

In [8]:
grading_source_cols = [
    ('Miinimumnõuded', 'version__grading__grade_preconditions__et'),
    ('Hindamismeetod', 'version__grading__grade_evaluation__et')
    # lisa veel, kui vaja midagi
]

def combine_grading_info(row):
    return None

df_resolved['hindamine_info'] = df_resolved.apply(combine_grading_info, axis=1)
print("Samm 4: Hindamisinfo koondatud.")

Samm 4: Hindamisinfo koondatud.


### Samm 5: Lõplik viimistlus

Valime lõplikud veerud. 

In [9]:
final_cols = [
    # Põhiinfo
    'aine_kood',         # code
    'nimi_et',           # title (eesti)
    'nimi_en',           # title (inglise)
    'eap',               # EAP / credits
    'kestus_sem',        # aine kestus semestrites
    'semester',          # kevad/sügis
    'oppeaste',          # bakalaureuse/magistri/doktori õpe
    'oppevorm_et',       # päevane/õhtune jne
    'oppekeel_et',       # õppekeel eesti
    'oppekeel_en',       # õppekeel inglise

    # Kirjeldus ja õppimise info
    'kirjeldus',         # overview__description
    'kirjeldus_en',      # overview__description_en
    'opivaljundid_et',   # learning outcomes eesti
    'opivaljundid_en',   # learning outcomes inglise

    # Hindamine
    'hindamine_info',    # ühendatud või eraldi hindamise info

    # Lisainfo
    'toimumisajad_json', # ajakava JSON
    'oppejoud_json',     # õppejõud JSON
    'linn',              # linna info
    'aine_tyyp_et'       # aine tüüp (tavaline, seminar jne)
]

existing_cols = [c for c in final_cols if c in df_resolved.columns]
df_final = df_resolved[existing_cols].copy()

print(f"Lõplik ridade arv: {len(df_final)}, lõplik veergude arv: {len(df_final.columns)}")
print("Samm 5: Lõplik andmestik koostatud.")

Lõplik ridade arv: 3416, lõplik veergude arv: 19
Samm 5: Lõplik andmestik koostatud.


### Samm 6: Salvestamine ja kontroll

Kontrollime mõningaid kvaliteediaspekte ning salvestame puhastatud faili.

* Salvesta puhastatud fail.
* Arvuta puuduvate väärtuste hulk igas veerus.
* Leia kategooriliste tunnustega veergude [keel, semester, õppeaste, …] enim levinud väärtused (näiteks df[tunnus].count().head(5))
* Näita "Sissejuhatus andmeteadusesse” aine kogu puhastatud kujul väljund lihtsasti loetaval kujul (võid küsida LLMilt abi, et väljund hästi kenasti vormistada, et see ei oleks lihtsalt välja prinditud andmerida).
* Lisada tunnus kirjeldus, sisaldab ühe aine koguinfot (kõik veerud ühendatud suureks tekstiks. Teha tunnuse kirjeldus tähemärkide arvu statistika (describe() funktsioon võib aidata, aga võib kasutada ka visualiseerimist), et teaksime, kui palju infot me RAGile ette anda kavatseme

In [14]:
import pandas as pd

# 1. Vali ainult lõplikud veerud
df_final = df_resolved[final_cols].copy()

# 2. Arvuta puuduvate väärtuste hulk igas veerus
print("Puuduvate väärtuste hulk veergude lõikes:")
print(df_final.isnull().sum())

# 3. Leia kategooriliste tunnustega veergude enim levinud väärtused
kategoorilised = ['oppekeel_et', 'oppekeel_en', 'semester', 'oppeaste', 'oppevorm_et', 'linn', 'aine_tyyp_et']
print("\nKategooriliste tunnuste sagedus:")
for col in kategoorilised:
    if col in df_final.columns:
        print(f"\nVeerg '{col}':")
        print(df_final[col].value_counts().head(5))

# 4. Näita "Sissejuhatus andmeteadusesse" aine kogu puhastatud kujul
aine_nimi = "Sissejuhatus andmeteadusse"
aine_row = df_final[df_final['nimi_et'] == aine_nimi]
if not aine_row.empty:
    print(f"\nAine '{aine_nimi}' kogu info:\n")
    # Ühenda kõik veerud üheks tekstiks
    aine_text = "\n".join(aine_row.iloc[0].astype(str).tolist())
    print(aine_text)
else:
    print(f"\nAinet '{aine_nimi}' andmestikus ei leitud.")

# 5. Lisa tunnus 'kirjeldus', mis sisaldab kogu aine infot ühe tekstina
df_final['kirjeldus'] = df_final.apply(lambda row: "\n".join(row.astype(str).tolist()), axis=1)

# 6. Tähemärkide arvu statistika kirjelduses
print("\nKirjeldusveeru tähemärkide arvu statistika:")
df_final['kirjeldus_length'] = df_final['kirjeldus'].apply(len)
print(df_final['kirjeldus_length'].describe())

# 7. Salvesta puhastatud fail
OUTPUT_FILE = "andmed_puhastatud.csv"
df_final.to_csv(OUTPUT_FILE, index=False)
print(f"\nSamm 6: Andmestik salvestatud faili '{OUTPUT_FILE}'.")

Puuduvate väärtuste hulk veergude lõikes:
aine_kood               0
nimi_et                 0
nimi_en                 0
eap                     0
kestus_sem              0
semester               17
oppeaste               67
oppevorm_et            17
oppekeel_et            17
oppekeel_en            17
kirjeldus               0
kirjeldus_en            0
opivaljundid_et      2003
opivaljundid_en         1
hindamine_info       3416
toimumisajad_json     604
oppejoud_json          18
linn                 1210
aine_tyyp_et            0
dtype: int64

Kategooriliste tunnuste sagedus:

Veerg 'oppekeel_et':
oppekeel_et
eesti keel      2555
inglise keel     844
Name: count, dtype: int64

Veerg 'oppekeel_en':
oppekeel_en
Estonian    2555
English      844
Name: count, dtype: int64

Veerg 'semester':
semester
kevad    1898
sügis    1501
Name: count, dtype: int64

Veerg 'oppeaste':
oppeaste
[{"code": "bachelor", "en": "bachelor's studies", "et": "bakalaureuseõpe"}]                                    